# Electric Consumption and Cost

Data is retrieved from [NYC Open Data](https://data.cityofnewyork.us/Housing-Development/Electric-Consumption-And-Cost-2010-April-2020-/jr24-e7cr).

This project is divided into the following:
* Data Preparation
* Testing & Training
* Target Variable
* Algorithm 1 - Default
* Algorithm 1 - Parameters
* Algorithm 2 - Default
* Algorithm 2 - Parameters

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from pandas.plotting import scatter_matrix

import mglearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [70]:
electric_consumption = pd.read_csv("/Users/sheryl/GitHub/data71200su21/Project 1/dataset/electric-consumption.csv",  
                                   low_memory=False)

In [71]:
electric_consumption

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800,7387.97,216.00,2808.00,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200,6607.87,224.00,2912.00,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200,5919.55,216.00,2808.00,5177.43
3,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14764.04,GOV/NYC/068,Yes,105600,6057.22,208.00,2704.00,6002.82
4,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13729.54,GOV/NYC/068,Yes,97600,5598.34,216.00,2808.00,5323.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301312,WYCKOFF GARDENS,BROOKLYN,WYCKOFF GARDENS,BLD 02,INTERVAL,NaN,163.0,272,K016300,FEDERAL,...,1096666,N,1287.71,GOV/NYC/068,Yes,0,0.00,116.16,1160.44,127.27
301313,WYCKOFF GARDENS,BROOKLYN,WYCKOFF GARDENS,BLD 03,INTERVAL,NaN,163.0,272,K016300,FEDERAL,...,1096667,N,1705.42,GOV/NYC/068,Yes,0,0.00,153.84,1536.86,168.56
301314,WYCKOFF GARDENS,BROOKLYN,WYCKOFF GARDENS,BLD 03,AMR,NaN,163.0,272,K016300,FEDERAL,...,8096664,N,3782.25,GOV/NYC/068,Yes,101200,3408.42,0.00,0.00,373.83
301315,WYCKOFF GARDENS,BROOKLYN,WYCKOFF GARDENS,BLD 02,AMR,NaN,163.0,272,K016300,FEDERAL,...,8096666,N,2541.43,GOV/NYC/068,Yes,68000,2290.24,0.00,0.00,251.19


In [72]:
electric_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301317 entries, 0 to 301316
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Development Name    301193 non-null  object 
 1   Borough             301317 non-null  object 
 2   Account Name        301317 non-null  object 
 3   Location            293386 non-null  object 
 4   Meter AMR           301231 non-null  object 
 5   Meter Scope         15799 non-null   object 
 6   TDS #               299936 non-null  float64
 7   EDP                 301317 non-null  int64  
 8   RC Code             301317 non-null  object 
 9   Funding Source      301317 non-null  object 
 10  AMP #               299972 non-null  object 
 11  Vendor Name         301317 non-null  object 
 12  UMIS BILL ID        301317 non-null  int64  
 13  Revenue Month       301317 non-null  object 
 14  Service Start Date  301314 non-null  object 
 15  Service End Date    301314 non-nul

In [73]:
electric_consumption.Borough.unique()

array(['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND',
       'NON DEVELOPMENT FACILITY', 'FHA'], dtype=object)

In [74]:
electric_consumption.describe()

,TDS #,EDP,UMIS BILL ID,# days,Current Charges,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
count,299936.000000,301317.000000,3.013170e+05,301314.000000,301317.000000,3.013170e+05,301317.000000,301317.000000,301317.000000,301317.000000
mean,118.513576,424.835111,4.471668e+06,30.365592,4359.692793,3.243415e+04,1586.340925,65.953820,1035.045797,1738.306071
std,128.908939,185.714308,1.963247e+06,1.664890,6237.267626,5.053216e+04,2649.869237,645.758963,1734.666074,3626.491037
min,1.000000,63.000000,1.726303e+06,1.000000,-1207.550000,0.000000e+00,0.000000,0.000000,0.000000,-59396.430000
25%,33.000000,227.000000,2.843192e+06,29.000000,446.130000,0.000000e+00,0.000000,0.000000,0.000000,30.380000
50%,64.000000,426.000000,4.100110e+06,30.000000,2573.780000,1.604000e+04,730.020000,27.220000,381.720000,1037.400000
75%,162.000000,552.000000,5.736544e+06,32.000000,5844.540000,4.784000e+04,2263.150000,94.400000,1528.060000,2780.500000
max,582.000000,999.000000,8.893530e+06,257.000000,329800.370000,1.779600e+06,195575.860000,329074.000000,78782.960000,134224.510000


In [75]:
electric_consumption.hist

<bound method hist_frame of        Development Name   Borough     Account Name Location Meter AMR  \
0                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
1                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
2                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
3                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
4                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
...                 ...       ...              ...      ...       ...   
301312  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 02  INTERVAL   
301313  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03  INTERVAL   
301314  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03       AMR   
301315  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 02       AMR   
301316  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03       AMR   

         Meter Scope  TDS #  EDP  RC Code Funding Source  ... Meter Number  \
0       BLD 01 to

In [76]:
electric_consumption_drop0 = electric_consumption[electric_consumption != 0]
print(electric_consumption_drop0)

       Development Name   Borough     Account Name Location Meter AMR  \
0                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
1                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
2                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
3                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
4                 ADAMS     BRONX            ADAMS   BLD 05      NONE   
...                 ...       ...              ...      ...       ...   
301312  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 02  INTERVAL   
301313  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03  INTERVAL   
301314  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03       AMR   
301315  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 02       AMR   
301316  WYCKOFF GARDENS  BROOKLYN  WYCKOFF GARDENS   BLD 03       AMR   

         Meter Scope  TDS #  EDP  RC Code Funding Source  ... Meter Number  \
0       BLD 01 to 07  118.0  248  B011800    

In [85]:
electric_consumption_dropcolumns = electric_consumption_drop0.drop(["UMIS BILL ID", "Consumption (KW)", "KW Charges", "Other charges", 
                               "Bill Analyzed", "Meter Number", "AMP #", "Account Name", "Service Start Date",
                              "Service End Date", "RC Code", "Development Name", "Location", "Meter AMR", "Meter Scope",
                              "Vendor Name", "Funding Source", "Rate Class", "Revenue Month", "Borough", "Estimated"], axis=1)
print(electric_consumption_dropcolumns)

        TDS #  EDP  # days  Current Charges  Consumption (KWH)  KWH Charges
0       118.0  248    33.0         15396.82           128800.0      7387.97
1       118.0  248    30.0         14556.34           115200.0      6607.87
2       118.0  248    29.0         13904.98           103200.0      5919.55
3       118.0  248    31.0         14764.04           105600.0      6057.22
4       118.0  248    28.0         13729.54            97600.0      5598.34
...       ...  ...     ...              ...                ...          ...
301312  163.0  272    29.0          1287.71                NaN          NaN
301313  163.0  272    29.0          1705.42                NaN          NaN
301314  163.0  272    29.0          3782.25           101200.0      3408.42
301315  163.0  272    29.0          2541.43            68000.0      2290.24
301316  163.0  272    29.0          2967.49            79400.0      2674.19

[301317 rows x 6 columns]


In [86]:
electric_consumption_new = electric_consumption_dropcolumns.dropna()
print(electric_consumption_new)

        TDS #  EDP  # days  Current Charges  Consumption (KWH)  KWH Charges
0       118.0  248    33.0         15396.82           128800.0      7387.97
1       118.0  248    30.0         14556.34           115200.0      6607.87
2       118.0  248    29.0         13904.98           103200.0      5919.55
3       118.0  248    31.0         14764.04           105600.0      6057.22
4       118.0  248    28.0         13729.54            97600.0      5598.34
...       ...  ...     ...              ...                ...          ...
301309  163.0  272    29.0          5608.58            67600.0      2276.77
301310  163.0  272    29.0          6438.24            77600.0      2613.57
301314  163.0  272    29.0          3782.25           101200.0      3408.42
301315  163.0  272    29.0          2541.43            68000.0      2290.24
301316  163.0  272    29.0          2967.49            79400.0      2674.19

[203466 rows x 6 columns]


In [89]:
electric_consumption_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203466 entries, 0 to 301316
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   TDS #              203466 non-null  float64
 1   EDP                203466 non-null  int64  
 2   # days             203466 non-null  float64
 3   Current Charges    203466 non-null  float64
 4   Consumption (KWH)  203466 non-null  float64
 5   KWH Charges        203466 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 10.9 MB


In [90]:
electric_consumption_new.describe()

,TDS #,EDP,# days,Current Charges,Consumption (KWH),KWH Charges
count,203466.000000,203466.000000,203466.000000,203466.000000,2.034660e+05,203466.000000
mean,143.322717,435.534748,30.365791,5750.880088,4.737377e+04,2314.152587
std,136.602664,186.697313,1.671624,6792.495794,5.446195e+04,2888.915812
min,1.000000,200.000000,1.000000,-1207.550000,1.000000e+00,0.050000
25%,43.000000,241.000000,29.000000,1514.360000,1.400000e+04,644.217500
50%,79.000000,429.000000,30.000000,4131.925000,3.520000e+04,1668.550000
75%,247.000000,566.000000,32.000000,7575.947500,6.248000e+04,3006.330000
max,582.000000,871.000000,154.000000,329800.370000,1.779600e+06,195575.860000


In [100]:
electric_consumption_new.corr()

,TDS #,EDP,# days,Current Charges,Consumption (KWH),KWH Charges
TDS #,1.000000,0.400172,-0.002514,-0.202685,-0.238400,-0.216062
EDP,0.400172,1.000000,0.000945,-0.227034,-0.235784,-0.214761
# days,-0.002514,0.000945,1.000000,0.035539,0.031634,0.058095
Current Charges,-0.202685,-0.227034,0.035539,1.000000,0.932690,0.922657
Consumption (KWH),-0.238400,-0.235784,0.031634,0.932690,1.000000,0.942372
KWH Charges,-0.216062,-0.214761,0.058095,0.922657,0.942372,1.000000


In [99]:
print(electric_consumption_new['Consumption (KWH)'].value_counts(ascending=False))

33600.0     391
60000.0     369
30240.0     351
64800.0     342
31200.0     335
           ... 
638000.0      1
698400.0      1
35820.0       1
5016.0        1
504000.0      1
Name: Consumption (KWH), Length: 7003, dtype: int64


In [114]:
print(electric_consumption_new['EDP'].value_counts(ascending=False))

650    4171
200    3440
214    3252
212    3013
369    3004
       ... 
324      60
835      48
257      47
395      24
340      12
Name: EDP, Length: 331, dtype: int64


# Visualizations

# Missing Value and Data Cleaning

# Training and Test Set

In [103]:
X = electric_consumption_new.drop(["TDS #", "EDP", "# days", "Consumption (KWH)"],axis=1)
print(X)

        Current Charges  KWH Charges
0              15396.82      7387.97
1              14556.34      6607.87
2              13904.98      5919.55
3              14764.04      6057.22
4              13729.54      5598.34
...                 ...          ...
301309          5608.58      2276.77
301310          6438.24      2613.57
301314          3782.25      3408.42
301315          2541.43      2290.24
301316          2967.49      2674.19

[203466 rows x 2 columns]


In [105]:
y = electric_consumption_new["Consumption (KWH)"]
print(y)

0         128800.0
1         115200.0
2         103200.0
3         105600.0
4          97600.0
            ...   
301309     67600.0
301310     77600.0
301314    101200.0
301315     68000.0
301316     79400.0
Name: Consumption (KWH), Length: 203466, dtype: float64


In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [115]:
from sklearn.linear_model import LinearRegression

linreg_train = LinearRegression().fit(X_train, y_train)
linreg_test = LinearRegression().fit(X_test, y_test)

In [116]:
print("Test set score: {:.2f}".format(linreg.score(X_test, y_test)))
print("Test set score: {:.2f}".format(linreg.score(X_train, y_train)))

Test set score: 0.92
Test set score: 0.91


# Data Transformations

In [117]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
print("knn score: {}".format(knn.score(X_train, y_train)))

knn score: 0.31796775282052786
